## Issue description

Currently Narwhals can read struct fields, but **can't create struct columns**.

I need to create struct fields with Narwhals for a feature I'm working on.

Currently Narwhals can read struct fields, but **can't create struct columns**.

I need to create struct fields with Narwhals for a feature I'm working on.

## Comment from MarcoGorelli

`concat_str might` not be miles off then? it should be simpler though, because:
- you wouldn't need to worry about doing casts
- you wouldn't need the extra keyword arguments like separator and ignore_nulls

## Investigating concepts

In dataframe systems like Polars, Apache Arrow, and Narwhals, a **struct** is a column where each value is like a small dictionary or object, containing multiple named fields.

#### Definition in Narwhals
Struct

Bases: NestedType

Struct composite type.

In Narwhals, **NestedType** is the base class for data types that contain nested elements — meaning values that aren’t just flat scalars like int or str, but instead have internal structure.

Each row in a **struct column** behaves like a small dictionary, but with a **known schema** and **fixed data types** for each field.

In [8]:
# Struct column "s" with 2 fields: a, b
import pandas as pd

df = pd.DataFrame({
    "my_struct_column": [
        {"a": 1, "b": "x"},
        {"a": 2, "b": "y"},
    ]
})

print(df.dtypes)

,my_struct_column
0,"{'a': 1, 'b': 'x'}"
1,"{'a': 2, 'b': 'y'}"


In [16]:
my_struct = [
        {"a": 1, "b": "x"},
        {"a": 2, "b": "y"},
    ]
wrong = pd.DataFrame( my_struct )
print(wrong)

   a  b
0  1  x
1  2  y


In [ ]:
from dataclasses import dataclass, make_dataclass
import pandas as pd
import pyarrow as pa
import polars as pl


data = {
    "a": [1, 2],
    "b": ["x", "y"]
}
wrong = pd.DataFrame( data )
print("direct, would create expanded dataframe:
", wrong)

class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __repr__(self):
        return f"(a={self.a}, b={self.b})"

candidate = pd.DataFrame( [ MyClass(**row) for _, row in wrong.iterrows() ] )
print("list of structs, (is this correct?):\n", candidate)

# MyDataClass = make_dataclass("MyClass", [("a", int), ("b", str)])
@dataclass
class MyDataClass:
    a: int
    b: str

candidate = pd.DataFrame( [ MyDataClass(**row) for _, row in wrong.iterrows() ] )
print("list of dataclass-structs, the dataframe gets expanded:\n", candidate)

is_this_correct = pd.DataFrame( [ {"struct": MyDataClass(**row)} for _, row in wrong.iterrows() ] )
print("wrapping the struct into a dict element (is this the expected output?):\n", is_this_correct)



direct, would create expanded dataframe:
    a  b
0  1  x
1  2  y
list of structs, (is this correct?):
             0
0  (a=1, b=x)
1  (a=2, b=y)
list of dataclass-structs, the dataframe gets expanded:
    a  b
0  1  x
1  2  y
wrapping the struct into a dict element (is this the expected output?):
                     struct
0  MyDataClass(a=1, b='x')
1  MyDataClass(a=2, b='y')


In [2]:
import polars as pl

df = pl.DataFrame({
    "a": [1, 2, 3],
    "b": ["x", 2, "z"]
}, strict=False)
print(df)
# Try to create a struct column from mixed dtypes
try:
    df.select(
        pl.struct(["a", "b"]).alias("my_struct")
    )
except Exception as e:
    print(repr(e))
df

shape: (3, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ str │
╞═════╪═════╡
│ 1   ┆ x   │
│ 2   ┆ 2   │
│ 3   ┆ z   │
└─────┴─────┘


a,b
i64,str
1,"""x"""
2,"""2"""
3,"""z"""


In [ ]:
nums = (2*x for x in [0, 1, 2, 3])

In [52]:
nums = [2*x for x in [0, 1, 2, 3, 4]]
nums

[0, 2, 4, 6, 8]